In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import csv
import os
import sys
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [3]:
# list working dir

print(os.getcwd())

C:\Users\micha\OneDrive - České vysoké učení technické v Praze\Dokumenty\PycharmProjects\optimizin\General-Optimization-Solver\src\vrp


In [5]:
# renaming results


def process_json_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                data = json.load(file)
                
            run_history = data.pop('run_history')
            
            for run in run_history:
                if 'solver_type' in run.keys() and run['solver_type'] == 'GA':
                    run['solution_progress'] = [[run['solution_value']*10, run['solve_time'], -1]]

            # Seřadíme záznamy ve slovníku
            data = {
                'instance_kind': 'VRPTW',
                'benchmark_name': 'solomon_100',
                'instance_name': data.pop('instance_name'),  # Consistency with original source of Solomon instances
                'reference_solution': data.pop('reference_solution'),
                'data': data.pop('data'),
                'run_history': run_history,
            }
            
            # Uložíme změny zpět do JSON souboru
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
        # break

folder_path = "..\\..\\data\\VRPTW\\solomon_100\\"
process_json_files(folder_path)


In [27]:
# merge results

folder = "..\\..\\output\\"

subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
print(subfolders)

# Ask user to choose the target folder

for i, folder in enumerate(subfolders):
    print(f"{i}: {folder}")

print()

target_folder = subfolders[int(input("Choose target folder: "))]

# use the first subfolder as target and copy the last results to them

for benchmark in os.scandir(subfolders[-1]):
    if benchmark.is_dir():
        print(benchmark.path)
        for file in os.scandir(benchmark):
            file_path = file.path
            if file_path.endswith('.json'):
                print(file_path)
                with open(file, 'r') as f:
                    target = json.load(f)
                for folder in subfolders[:-1]:
                    if not os.path.exists(os.path.join(folder, benchmark.name, file.name)):
                        continue
                    with open(os.path.join(folder, benchmark.name, file.name), 'r') as f:
                        source = json.load(f)
                    new = source['run_history'][-1]
                    target['run_history'].append(new)
                with open(file_path, 'w') as f:
                    json.dump(target, f, indent=4)
                    

['..\\..\\output\\1', '..\\..\\output\\2']
0: ..\..\output\1
1: ..\..\output\2

..\..\output\2\my_output
..\..\output\2\solomon_100
..\..\output\2\solomon_100\C101.json
..\..\output\2\solomon_100\C102.json
..\..\output\2\solomon_100\C103.json
..\..\output\2\solomon_100\C104.json
..\..\output\2\solomon_100\C105.json
..\..\output\2\solomon_100\C106.json
..\..\output\2\solomon_100\C107.json
..\..\output\2\solomon_100\C108.json
..\..\output\2\solomon_100\C109.json
..\..\output\2\solomon_100\C201.json
..\..\output\2\solomon_100\C202.json
..\..\output\2\solomon_100\C203.json
..\..\output\2\solomon_100\C204.json
..\..\output\2\solomon_100\C205.json
..\..\output\2\solomon_100\C206.json
..\..\output\2\solomon_100\C207.json
..\..\output\2\solomon_100\C208.json
..\..\output\2\solomon_100\R101.json
..\..\output\2\solomon_100\R102.json
..\..\output\2\solomon_100\R103.json
..\..\output\2\solomon_100\R104.json
..\..\output\2\solomon_100\R105.json
..\..\output\2\solomon_100\R106.json
..\..\output\2\so

In [16]:
# randomly select a few instances

import random
import os

folder = "..\\..\\data\\VRPTW\\"
instances = []
for subfolder in os.listdir(folder):
    if os.path.isdir(os.path.join(folder, subfolder)):
        for file in os.listdir(os.path.join(folder, subfolder)):
            if file.endswith(".json"):
                instances.append(os.path.join(folder, subfolder, file))
                
# random.seed(0)
train = random.sample(instances, 4)
print(train)

['..\\..\\data\\VRPTW\\solomon_100\\RC108.json', '..\\..\\data\\VRPTW\\solomon_50\\RC202.json', '..\\..\\data\\VRPTW\\solomon_25\\R103.json', '..\\..\\data\\VRPTW\\solomon_100\\R202.json']


In [17]:
test = random.sample([instance for instance in instances if instance not in train], 5)
print(test)

['..\\..\\data\\VRPTW\\solomon_25\\R204.json', '..\\..\\data\\VRPTW\\solomon_50\\R103.json', '..\\..\\data\\VRPTW\\solomon_25\\RC208.json', '..\\..\\data\\VRPTW\\solomon_50\\R104.json', '..\\..\\data\\VRPTW\\solomon_100\\RC203.json']


In [20]:
from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
# optimize hyperparameters

from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.algorithms.hyperparameters import HyperparameterProblem, MultiRun, stats_avg_nevals
from pymoo.algorithms.soo.nonconvex.g3pcx import G3PCX
from pymoo.core.mixed import MixedVariableGA
from pymoo.core.parameters import set_params, hierarchical
from pymoo.core.termination import TerminateIfAny
from pymoo.optimize import minimize
from pymoo.problems.single import Sphere
from pymoo.termination.fmin import MinimumFunctionValueTermination
from pymoo.termination.max_eval import MaximumFunctionCallTermination
from src.vrp.solvers.ga_model import *
from src.vrp.problem import *

algorithm = DE()
# algorithm = BRKGA()

class VRPTW(ElementwiseProblem):
    """pymoo wrapper class
    """

    def __init__(self, instance, fitness_func_):
        # print("number of customers", instance.nb_customers)
        super().__init__(n_var=instance.nb_customers, n_obj=1, xl=0, xu=1)
        self.instance = instance
        self.fitness_func = fitness_func_

    def _evaluate(self, x, out, *args, **kwargs):
        out = self.fitness_func(self.instance, x, out)

        assert "solution" not in out, "Do not use `solution` key, it is pymoo reserved keyword"

        return out
            
hyperparams = []
            
for instance in train:
    print(instance)
    instance = load_instance(instance)
    problem = VRPTW(instance, fitness_func)
    
    termination = TerminateIfAny(MinimumFunctionValueTermination(1e-5), MaximumFunctionCallTermination(500))
    
    performance = MultiRun(problem, seeds=[5, 50, 500], func_stats=stats_avg_nevals, termination=termination)
    
    res = minimize(HyperparameterProblem(algorithm, performance),
                   MixedVariableGA(pop_size=5),
                   ("time", "00:5:00"),
                   seed=1,
                   verbose=False)
    
    print(res.X)
    
    hyperparams.append(res.X.copy())
    
print()
print(hyperparams)

..\..\data\VRPTW\solomon_100\RC108.json
{'mating.jitter': False, 'mating.CR': 0.4502132028215444, 'mating.crossover': 'bin', 'mating.F': 0.4277015784306394, 'mating.n_diffs': 1, 'mating.selection': 'rand'}
..\..\data\VRPTW\solomon_50\RC202.json
{'mating.jitter': False, 'mating.CR': 0.4502132028215444, 'mating.crossover': 'bin', 'mating.F': 0.4277015784306394, 'mating.n_diffs': 1, 'mating.selection': 'rand'}
..\..\data\VRPTW\solomon_25\R103.json
{'mating.jitter': False, 'mating.CR': 0.4502132028215444, 'mating.crossover': 'bin', 'mating.F': 0.4277015784306394, 'mating.n_diffs': 1, 'mating.selection': 'rand'}
..\..\data\VRPTW\solomon_100\R202.json
{'mating.jitter': False, 'mating.CR': 0.4502132028215444, 'mating.crossover': 'bin', 'mating.F': 0.4277015784306394, 'mating.n_diffs': 1, 'mating.selection': 'rand'}

[{'mating.jitter': False, 'mating.CR': 0.4502132028215444, 'mating.crossover': 'bin', 'mating.F': 0.4277015784306394, 'mating.n_diffs': 1, 'mating.selection': 'rand'}, {'mating.ji

In [8]:
# save hyperparameters

with open("hyperparameters.json", "w") as f:
    json.dump(hyperparams, f, indent=4)

TypeError: Object of type bool_ is not JSON serializable

In [ ]:
# load hyperparameters

with open("hyperparameters.json", "r") as f:
    hyperparams = json.load(f)

In [21]:
# evaluate on test instances

results = np.zeros((len(test), len(hyperparams) + 1))


for i, instance in enumerate(test):
    print(i)
    instance = load_instance(instance)
    
    # Add default hyperparameters
    algorithm = DE()
    # algorithm = BRKGA()
    problem = VRPTW(instance, fitness_func)
    res = minimize(problem, algorithm, termination=("time", "00:1:00"), seed=1, verbose=False)
    results[i, 0] = res.F[0]
    
    for j, hyperparam in enumerate(hyperparams):
        algorithm = DE()
        # algorithm = BRKGA()
        set_params(algorithm, hierarchical(hyperparam))
        problem = VRPTW(instance, fitness_func)
        res = minimize(problem, algorithm, termination=("time", "00:1:00"), seed=1, verbose=False)
        results[i, j+1] = res.F[0]
        

0
1
2
3
4


In [22]:
# find the best hyperparameters

best_hyperparams = np.argmin(np.mean(results, axis=0))
print(best_hyperparams)
print(hyperparams[best_hyperparams])

0
{'mating.jitter': False, 'mating.CR': 0.4502132028215444, 'mating.crossover': 'bin', 'mating.F': 0.4277015784306394, 'mating.n_diffs': 1, 'mating.selection': 'rand'}
